In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hopsworks
import src.config as config

In [ ]:
# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
# Create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [ ]:
ts_data, _ = feature_view.training_data(description='Time-series hourly taxi rides')

2023-05-23 00:18:02,416 INFO: USE `taxi_demand_ytjqdg_featurestore`
2023-05-23 00:18:03,003 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_ytjqdg_featurestore`.`time_series_hourly_feature_group_1` `fg0`


In [ ]:
from src.paths import LOGS_DIR

ts_data_first_and_last_event = ts_data.agg({'pickup_hour': ['min', 'max']}).reset_index().rename(columns={'index': 'position'})
ts_data_first_and_last_event.to_csv(LOGS_DIR / 'logs_feature_pipeline.csv', index=False)